### Evaluation Sample Dataset to be sent to Lichess in PGN Format

In [0]:
%pip install python-chess

In [0]:
SERVING_ENDPOINT_NAME = "dbdemos_drug_extraction_fine_tuned_03_llama_3_2_3B_Instruct"
PARENT_CHESS_GPT = '/Volumes/unitygo/lichess/engine/chessgpt'

In [0]:
import io
import copy
import chess
import chess.pgn

In [0]:
def parse_lichess_pgn(pgn_content):
    """
    Parses a Lichess PGN file and returns structured data with headers, moves, comments, and game state.
    Returns a list of games (for files with multiple games).
    """
    games = []
    pgn = io.StringIO(pgn_content)
    while True:
        game = chess.pgn.read_game(pgn)
        if not game:
            break
        headers = dict(game.headers)
        moves = []
        board = game.board()
        node = game
        move_number = 1
        while node.variations:
            node = node.variation(0)
            move = node.move
            move_data = {
                "move_number": move_number,
                "turn": "white" if board.turn == chess.WHITE else "black",
                "san": board.san(move),
                "uci": move.uci(),
                "comment": node.comment,
                "fen_before": board.fen(),
            }
            try:
                move_data["nags"] = [chess.pgn.symbol_for_nag(nag) for nag in node.nags]
            except:
                move_data["nags"] = []
            # Update board state
            board.push(move)
            move_data["fen_after"] = board.fen()
            moves.append(move_data)
            move_number += (0 if board.turn == chess.BLACK else 1)  # Increment after black moves
        games.append({
            "headers": headers,
            "moves": moves,
            "termination": headers.get("Termination", ""),
            "result": headers.get("Result", "")
        })
    return games

structured_game_data = []

system_prompt = {"role": "system", "content": """
### INSTRUCTIONS:
You are a professional chess commentator. Your task is to provide an in-depth analysis of the following chess game moves and overall board state. Follow these guidelines:

1. Add detailed commentary related to the chess game moves, overall board state, player profile details like Elo rating, and tournament details if available, for example time control or game variant.
2. Explain the opening principles, key middlegame plans, and the final outcome. 
3. Explain what the player might be thinking with each move on the game.
4. Do not add any text before or after the list.
5. Not every move is supposed to have a commentary, make sure you sound as human as possible.
"""}

In [0]:
eval_file = f'{PARENT_CHESS_GPT}/lichess_pgn_2025.03.27_stockfish_vs_databricks-claude-3-7-sonnet.AM7d932b.pgn'
with open(eval_file, encoding='latin-1') as f:
    pgn_content = f.read()
    games = parse_lichess_pgn(pgn_content)
    if games:
        structured_game_data = [games[0]]
eval_with_header = []
eval_moves_list_with_header = []
exclusion_header = ['Result','ECO','Opening','Termination','termination','result']
structured_game_data_updated = []
for item in structured_game_data:
    item['headers'].pop('Annotator',None)
    structured_game_data_updated.append(item)
eval_with_header = copy.deepcopy(structured_game_data_updated)
for item_2 in structured_game_data:
    for x in exclusion_header:
        item_2['headers'].pop(x,None)
    updated_moves_list = []
    for x in item_2['moves']:
        x.pop('comment',None)
        x.pop('nags',None)
        updated_moves_list.append(x)
    eval_moves_list_with_header.append({'headers':item_2['headers'],'moves':updated_moves_list})
eval_moves_list_with_header

In [0]:
eval_with_header

In [0]:
import io
import re
import json
import chess.pgn
from chess import Board
from typing import Dict, Any, Union
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, MapType

class ChessPGNConverter:
    @staticmethod
    def json_to_pgn(json_input: Union[str, Dict[str, Any]]) -> str:
        """
        Core conversion logic for chess PGN/JSON transformations
        """
        if isinstance(json_input, str):
            try:
                try:
                    json_data = json.loads(json_input)
                except json.JSONDecodeError:
                    fixed_json = re.sub(r"(?<!\\)'", '"', json_input)
                    json_data = json.loads(fixed_json)
            except json.JSONDecodeError as e:
                return f"ERROR: Invalid JSON - {str(e)}"
        elif isinstance(json_input, dict):
            json_data = json_input
        else:
            return "ERROR: Input must be JSON string or dictionary"
        try:
            game = chess.pgn.Game()
            headers = json_data.get("headers", {})
            for key, value in headers.items():
                game.headers[str(key)] = str(value) if value is not None else ""
            if "result" in json_data and "Result" not in game.headers:
                game.headers["Result"] = str(json_data["result"])
            node = game
            board = Board()
            for move_data in json_data.get("moves", []):
                san = move_data.get("san")
                if not san or not isinstance(san, str):
                    continue
                try:
                    move = board.parse_san(san.strip())
                    node = node.add_variation(move)
                    if "comment" in move_data and move_data["comment"] is not None:
                        node.comment = str(move_data["comment"])
                    for nag in move_data.get("nags", []):
                        try:
                            if nag:
                                node.nags.add(chess.pgn.nag_from_symbol(str(nag).strip()))
                        except ValueError:
                            pass
                    board.push(move)
                except ValueError:
                    continue
            exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
            return str(game.accept(exporter))
        except Exception as e:
            return f"ERROR: {str(e)}"
    @staticmethod
    def pgn_to_json(pgn_text: str) -> Dict[str, Any]:
        try:
            game = chess.pgn.read_game(io.StringIO(pgn_text))
            if not game:
                return {"error": "No valid PGN data found"}
            
            headers = dict(game.headers)
            moves = []
            board = game.board()
            node = game
            move_number = 1
            while not node.is_end():
                next_node = node.variation(0) if node.variations else None
                if not next_node:
                    break
                
                move = next_node.move
                move_data = {
                    "move_number": move_number,
                    "turn": "white" if board.turn == chess.WHITE else "black",
                    "san": board.san(move),
                    "uci": move.uci(),
                    "comment": next_node.comment,
                    "nags": [chess.pgn.symbol_for_nag(nag) for nag in next_node.nags],
                    "fen_before": board.fen(),
                }
                board.push(move)
                move_data["fen_after"] = board.fen()
                moves.append(move_data)
                move_number += (0 if board.turn == chess.BLACK else 1)
            return {
                "headers": headers,
                "moves": moves,
                "termination": headers.get("Termination", ""),
                "result": headers.get("Result", "")
                }
        except Exception as e:
            return {"error": str(e)}
json_to_pgn_udf = udf(ChessPGNConverter.json_to_pgn, StringType())
pgn_to_json_udf = udf(ChessPGNConverter.pgn_to_json, MapType(StringType(), StringType()))
# def json_to_pgn(json_input: Union[str, Dict[str, Any]]) -> str:
#     return ChessPGNConverter.json_to_pgn(json_input)
# def pgn_to_json(pgn_text: str) -> Dict[str, Any]:
#     return ChessPGNConverter.pgn_to_json(pgn_text)

In [0]:
from pyspark.sql import functions as F
data = [
    ("""{'headers': {'Event': 'LLM Tournament Match',
   'Site': '?',
   'Date': '????.??.??',
   'Round': '?',
   'White': 'stockfish',
   'Black': 'databricks-claude-3-7-sonnet',
   'Result': '1-0',
   'GameId': 'AM7d932b',
   'WhiteElo': '?',
   'BlackElo': '?',
   'Variant': 'Standard',
   'TimeControl': '-',
   'ECO': 'C57',
   'Opening': 'Italian Game: Two Knights Defense, Ponziani-Steinitz Gambit',
   'Termination': 'Unknown'},
  'moves': [{'move_number': 1,
    'turn': 'white',
    'san': 'e4',
    'uci': 'e2e4',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'},
   {'move_number': 1,
    'turn': 'black',
    'san': 'e5',
    'uci': 'e7e5',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2'},
   {'move_number': 2,
    'turn': 'white',
    'san': 'Nf3',
    'uci': 'g1f3',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2'},
   {'move_number': 2,
    'turn': 'black',
    'san': 'Nc6',
    'uci': 'b8c6',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2',
    'nags': [],
    'fen_after': 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3'},
   {'move_number': 3,
    'turn': 'white',
    'san': 'Bc4',
    'uci': 'f1c4',
    'comment': '',
    'fen_before': 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3',
    'nags': [],
    'fen_after': 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3'},
   {'move_number': 3,
    'turn': 'black',
    'san': 'Nf6',
    'uci': 'g8f6',
    'comment': '',
    'fen_before': 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4'},
   {'move_number': 4,
    'turn': 'white',
    'san': 'Ng5',
    'uci': 'f3g5',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n2n2/4p1N1/2B1P3/8/PPPP1PPP/RNBQK2R b KQkq - 5 4'},
   {'move_number': 4,
    'turn': 'black',
    'san': 'Nxe4',
    'uci': 'f6e4',
    'comment': 'C57 Italian Game: Two Knights Defense, Ponziani-Steinitz Gambit',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n2n2/4p1N1/2B1P3/8/PPPP1PPP/RNBQK2R b KQkq - 5 4',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n5/4p1N1/2B1n3/8/PPPP1PPP/RNBQK2R w KQkq - 0 5'},
   {'move_number': 5,
    'turn': 'white',
    'san': 'Bxf7+',
    'uci': 'c4f7',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n5/4p1N1/2B1n3/8/PPPP1PPP/RNBQK2R w KQkq - 0 5',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1Bpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R b KQkq - 0 5'},
   {'move_number': 5,
    'turn': 'black',
    'san': 'Ke7',
    'uci': 'e8e7',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1Bpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R b KQkq - 0 5',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R w KQ - 1 6'},
   {'move_number': 6,
    'turn': 'white',
    'san': 'd4',
    'uci': 'd2d4',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R w KQ - 1 6',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/3Pn3/8/PPP2PPP/RNBQK2R b KQ - 0 6'},
   {'move_number': 6,
    'turn': 'black',
    'san': 'Nxd4',
    'uci': 'c6d4',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/3Pn3/8/PPP2PPP/RNBQK2R b KQ - 0 6',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/8/PPP2PPP/RNBQK2R w KQ - 0 7'},
   {'move_number': 7,
    'turn': 'white',
    'san': 'c3',
    'uci': 'c2c3',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/8/PPP2PPP/RNBQK2R w KQ - 0 7',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 7'},
   {'move_number': 7,
    'turn': 'black',
    'san': 'Rg8',
    'uci': 'h8g8',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 7',
    'nags': [],
    'fen_after': 'r1bq1br1/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 8'},
   {'move_number': 8,
    'turn': 'white',
    'san': 'Bxg8',
    'uci': 'f7g8',
    'comment': '',
    'fen_before': 'r1bq1br1/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 8',
    'nags': [],
    'fen_after': 'r1bq1bB1/ppppk1pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 8'},
   {'move_number': 8,
    'turn': 'black',
    'san': 'Ke8',
    'uci': 'e7e8',
    'comment': '',
    'fen_before': 'r1bq1bB1/ppppk1pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 8',
    'nags': [],
    'fen_after': 'r1bqkbB1/pppp2pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 9'},
   {'move_number': 9,
    'turn': 'white',
    'san': 'Bf7+',
    'uci': 'g8f7',
    'comment': '',
    'fen_before': 'r1bqkbB1/pppp2pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 9',
    'nags': [],
    'fen_after': 'r1bqkb2/pppp1Bpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 2 9'},
   {'move_number': 9,
    'turn': 'black',
    'san': 'Ke7',
    'uci': 'e8e7',
    'comment': '',
    'fen_before': 'r1bqkb2/pppp1Bpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 2 9',
    'nags': [],
    'fen_after': 'r1bq1b2/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 3 10'},
   {'move_number': 10,
    'turn': 'white',
    'san': 'cxd4',
    'uci': 'c3d4',
    'comment': '',
    'fen_before': 'r1bq1b2/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 3 10',
    'nags': [],
    'fen_after': 'r1bq1b2/ppppkBpp/8/4p1N1/3Pn3/8/PP3PPP/RNBQK2R b KQ - 0 10'},
   {'move_number': 10,
    'turn': 'black',
    'san': 'Kd6',
    'uci': 'e7d6',
    'comment': '',
    'fen_before': 'r1bq1b2/ppppkBpp/8/4p1N1/3Pn3/8/PP3PPP/RNBQK2R b KQ - 0 10',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/3k4/4p1N1/3Pn3/8/PP3PPP/RNBQK2R w KQ - 1 11'},
   {'move_number': 11,
    'turn': 'white',
    'san': 'Nxe4+',
    'uci': 'g5e4',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/3k4/4p1N1/3Pn3/8/PP3PPP/RNBQK2R w KQ - 1 11',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/3k4/4p3/3PN3/8/PP3PPP/RNBQK2R b KQ - 0 11'},
   {'move_number': 11,
    'turn': 'black',
    'san': 'Kc6',
    'uci': 'd6c6',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/3k4/4p3/3PN3/8/PP3PPP/RNBQK2R b KQ - 0 11',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PP3PPP/RNBQK2R w KQ - 1 12'},
   {'move_number': 12,
    'turn': 'white',
    'san': 'Qc2+',
    'uci': 'd1c2',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PP3PPP/RNBQK2R w KQ - 1 12',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PPQ2PPP/RNB1K2R b KQ - 2 12'},
   {'move_number': 12,
    'turn': 'black',
    'san': 'Bc5',
    'uci': 'f8c5',
    'comment': 'White wins.',
    'fen_before': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PPQ2PPP/RNB1K2R b KQ - 2 12',
    'nags': [],
    'fen_after': 'r1bq4/pppp1Bpp/2k5/2b1p3/3PN3/8/PPQ2PPP/RNB1K2R w KQ - 3 13'}],
  'termination': 'Unknown',
  'result': '1-0'}""",),
    ('{"headers":{"Event":"Test"},"moves":[{"san":"d4","comment":"Queens Gambit"}]}',)
]
df = spark.createDataFrame(data, ["json_data"])
df = df.withColumn("pgn_output", json_to_pgn_udf(F.col("json_data")))
display(df)

In [0]:
%pip install unitycatalog-ai[databricks]

In [0]:
def json_to_pgn(json_input: str) -> str:
    """
    Converts JSON chess game data to PGN format with robust input handling.
    
    Args:
        json_input: Either:
            - A properly formatted JSON string (with double quotes)
            - A dictionary containing game data
    
    Returns:
        str: PGN formatted string
    
    Raises:
        ValueError: If input is invalid or parsing fails
    """
    import warnings
    warnings.filterwarnings("ignore") 
    import pip
    try:
        import chess.pgn
        from chess import Board
    except ImportError:
        pip.main(['install', 'python-chess'])
        import chess.pgn
        from chess import Board
    import json

    if isinstance(json_input, str):
        try:
            json_data = json.loads(json_input)
        except json.JSONDecodeError as e:
            try:
                json_input = json_input.replace("'",'"')
                json_data = json.loads(json_input)
            except:
                raise ValueError(f"Invalid JSON string: {e}") from e
    elif isinstance(json_input, dict):
        json_data = json_input
    else:
        raise ValueError("Input must be either JSON string or dictionary")
    
    if not isinstance(json_data, dict):
        raise ValueError("JSON data must be an object/dictionary")
    
    game = chess.pgn.Game()
    headers = json_data.get("headers", {})
    
    if not isinstance(headers, dict):
        raise ValueError("Headers must be a dictionary")
    
    for key, value in headers.items():
        game.headers[str(key)] = str(value) if value is not None else ""
    
    result = json_data.get("result")
    if result and "Result" not in game.headers:
        game.headers["Result"] = str(result)
    
    node = game
    board = Board()
    moves = json_data.get("moves", [])
    
    if not isinstance(moves, list):
        raise ValueError("Moves must be a list")
    
    for idx, move_data in enumerate(moves):
        if not isinstance(move_data, dict):
            raise ValueError(f"Move {idx + 1} must be a dictionary")
        
        try:
            san = move_data.get("san")
            if not san or not isinstance(san, str):
                raise ValueError(f"Move {idx + 1} missing valid SAN string")
            
            move = board.parse_san(san)
            node = node.add_variation(move)
            
            comment = move_data.get("comment")
            if comment is not None:
                node.comment = str(comment)
            
            nags = move_data.get("nags", [])
            if isinstance(nags, list):
                for nag_symbol in nags:
                    try:
                        if nag_symbol:
                            nag = chess.pgn.nag_from_symbol(str(nag_symbol))
                            node.nags.add(nag)
                    except (ValueError, TypeError):
                        pass
            
            board.push(move)
        except (ValueError, AttributeError) as e:
            raise ValueError(f"Error processing move {idx + 1} ({move_data.get('san')}): {str(e)}") from e
    
    try:
        exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
        return str(game.accept(exporter))
    except Exception as e:
        raise ValueError(f"PGN generation failed: {str(e)}") from e

json_string = """{'headers': {'Event': 'LLM Tournament Match',
   'Site': '?',
   'Date': '????.??.??',
   'Round': '?',
   'White': 'stockfish',
   'Black': 'databricks-claude-3-7-sonnet',
   'Result': '1-0',
   'GameId': 'AM7d932b',
   'WhiteElo': '?',
   'BlackElo': '?',
   'Variant': 'Standard',
   'TimeControl': '-',
   'ECO': 'C57',
   'Opening': 'Italian Game: Two Knights Defense, Ponziani-Steinitz Gambit',
   'Termination': 'Unknown'},
  'moves': [{'move_number': 1,
    'turn': 'white',
    'san': 'e4',
    'uci': 'e2e4',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'},
   {'move_number': 1,
    'turn': 'black',
    'san': 'e5',
    'uci': 'e7e5',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2'},
   {'move_number': 2,
    'turn': 'white',
    'san': 'Nf3',
    'uci': 'g1f3',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
    'nags': [],
    'fen_after': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2'},
   {'move_number': 2,
    'turn': 'black',
    'san': 'Nc6',
    'uci': 'b8c6',
    'comment': '',
    'fen_before': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2',
    'nags': [],
    'fen_after': 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3'},
   {'move_number': 3,
    'turn': 'white',
    'san': 'Bc4',
    'uci': 'f1c4',
    'comment': '',
    'fen_before': 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3',
    'nags': [],
    'fen_after': 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3'},
   {'move_number': 3,
    'turn': 'black',
    'san': 'Nf6',
    'uci': 'g8f6',
    'comment': '',
    'fen_before': 'r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 3 3',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4'},
   {'move_number': 4,
    'turn': 'white',
    'san': 'Ng5',
    'uci': 'f3g5',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n2n2/4p1N1/2B1P3/8/PPPP1PPP/RNBQK2R b KQkq - 5 4'},
   {'move_number': 4,
    'turn': 'black',
    'san': 'Nxe4',
    'uci': 'f6e4',
    'comment': 'C57 Italian Game: Two Knights Defense, Ponziani-Steinitz Gambit',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n2n2/4p1N1/2B1P3/8/PPPP1PPP/RNBQK2R b KQkq - 5 4',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1ppp/2n5/4p1N1/2B1n3/8/PPPP1PPP/RNBQK2R w KQkq - 0 5'},
   {'move_number': 5,
    'turn': 'white',
    'san': 'Bxf7+',
    'uci': 'c4f7',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1ppp/2n5/4p1N1/2B1n3/8/PPPP1PPP/RNBQK2R w KQkq - 0 5',
    'nags': [],
    'fen_after': 'r1bqkb1r/pppp1Bpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R b KQkq - 0 5'},
   {'move_number': 5,
    'turn': 'black',
    'san': 'Ke7',
    'uci': 'e8e7',
    'comment': '',
    'fen_before': 'r1bqkb1r/pppp1Bpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R b KQkq - 0 5',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R w KQ - 1 6'},
   {'move_number': 6,
    'turn': 'white',
    'san': 'd4',
    'uci': 'd2d4',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/4n3/8/PPPP1PPP/RNBQK2R w KQ - 1 6',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/3Pn3/8/PPP2PPP/RNBQK2R b KQ - 0 6'},
   {'move_number': 6,
    'turn': 'black',
    'san': 'Nxd4',
    'uci': 'c6d4',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/2n5/4p1N1/3Pn3/8/PPP2PPP/RNBQK2R b KQ - 0 6',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/8/PPP2PPP/RNBQK2R w KQ - 0 7'},
   {'move_number': 7,
    'turn': 'white',
    'san': 'c3',
    'uci': 'c2c3',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/8/PPP2PPP/RNBQK2R w KQ - 0 7',
    'nags': [],
    'fen_after': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 7'},
   {'move_number': 7,
    'turn': 'black',
    'san': 'Rg8',
    'uci': 'h8g8',
    'comment': '',
    'fen_before': 'r1bq1b1r/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 7',
    'nags': [],
    'fen_after': 'r1bq1br1/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 8'},
   {'move_number': 8,
    'turn': 'white',
    'san': 'Bxg8',
    'uci': 'f7g8',
    'comment': '',
    'fen_before': 'r1bq1br1/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 8',
    'nags': [],
    'fen_after': 'r1bq1bB1/ppppk1pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 8'},
   {'move_number': 8,
    'turn': 'black',
    'san': 'Ke8',
    'uci': 'e7e8',
    'comment': '',
    'fen_before': 'r1bq1bB1/ppppk1pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 0 8',
    'nags': [],
    'fen_after': 'r1bqkbB1/pppp2pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 9'},
   {'move_number': 9,
    'turn': 'white',
    'san': 'Bf7+',
    'uci': 'g8f7',
    'comment': '',
    'fen_before': 'r1bqkbB1/pppp2pp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 1 9',
    'nags': [],
    'fen_after': 'r1bqkb2/pppp1Bpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 2 9'},
   {'move_number': 9,
    'turn': 'black',
    'san': 'Ke7',
    'uci': 'e8e7',
    'comment': '',
    'fen_before': 'r1bqkb2/pppp1Bpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R b KQ - 2 9',
    'nags': [],
    'fen_after': 'r1bq1b2/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 3 10'},
   {'move_number': 10,
    'turn': 'white',
    'san': 'cxd4',
    'uci': 'c3d4',
    'comment': '',
    'fen_before': 'r1bq1b2/ppppkBpp/8/4p1N1/3nn3/2P5/PP3PPP/RNBQK2R w KQ - 3 10',
    'nags': [],
    'fen_after': 'r1bq1b2/ppppkBpp/8/4p1N1/3Pn3/8/PP3PPP/RNBQK2R b KQ - 0 10'},
   {'move_number': 10,
    'turn': 'black',
    'san': 'Kd6',
    'uci': 'e7d6',
    'comment': '',
    'fen_before': 'r1bq1b2/ppppkBpp/8/4p1N1/3Pn3/8/PP3PPP/RNBQK2R b KQ - 0 10',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/3k4/4p1N1/3Pn3/8/PP3PPP/RNBQK2R w KQ - 1 11'},
   {'move_number': 11,
    'turn': 'white',
    'san': 'Nxe4+',
    'uci': 'g5e4',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/3k4/4p1N1/3Pn3/8/PP3PPP/RNBQK2R w KQ - 1 11',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/3k4/4p3/3PN3/8/PP3PPP/RNBQK2R b KQ - 0 11'},
   {'move_number': 11,
    'turn': 'black',
    'san': 'Kc6',
    'uci': 'd6c6',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/3k4/4p3/3PN3/8/PP3PPP/RNBQK2R b KQ - 0 11',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PP3PPP/RNBQK2R w KQ - 1 12'},
   {'move_number': 12,
    'turn': 'white',
    'san': 'Qc2+',
    'uci': 'd1c2',
    'comment': '',
    'fen_before': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PP3PPP/RNBQK2R w KQ - 1 12',
    'nags': [],
    'fen_after': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PPQ2PPP/RNB1K2R b KQ - 2 12'},
   {'move_number': 12,
    'turn': 'black',
    'san': 'Bc5',
    'uci': 'f8c5',
    'comment': 'White wins.',
    'fen_before': 'r1bq1b2/pppp1Bpp/2k5/4p3/3PN3/8/PPQ2PPP/RNB1K2R b KQ - 2 12',
    'nags': [],
    'fen_after': 'r1bq4/pppp1Bpp/2k5/2b1p3/3PN3/8/PPQ2PPP/RNB1K2R w KQ - 3 13'}],
  'termination': 'Unknown',
  'result': '1-0'}"""

pgn_from_string = json_to_pgn(json_string)
print("From string input:\n", pgn_from_string)

In [0]:

# def json_to_pgn(json_input: str) -> str:
#     """Function to convert the json output from model to pgn format. 
#     json_input: json output or dictionary in string format given as output from the llm that needs to be converted to pgn format.
#     """
#     return json_to_pgn_output(json_input)

In [0]:
json_to_pgn(json_string)

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
client = DatabricksFunctionClient()
CATALOG = "unitygo"
SCHEMA = "lichess"
function_info = client.create_python_function(
  func=json_to_pgn,
  catalog=CATALOG,
  schema=SCHEMA,
)

In [0]:
%sql
DROP FUNCTION IF EXISTS unitygo.lichess.json_to_pgn;